# Thai Character Cluster based on Dr. Virach Sornlertlamvanich 

In [1]:
import re
import os

In [2]:
dataset = os.listdir('encyclopedia')
sentences = []
targets = []
with open(os.path.join('encyclopedia', dataset[13]), 'r') as file:
    for line in file:
        line = line.replace("\n","")
        sentences.append(line.replace("|",""))
        targets.append(line.split("|"))
    

In [3]:
sentences

['\ufeffอาหาร',
 'เด็กๆ คงเคยเห็นวัวควายเดินกินหญ้าอย่างช้าๆ อยู่ในทุ่งนาเห็นม้ายืนเล็มหญ้าอยู่ในสนามหญ้าสีเขียว',
 'เห็นแม่ไก่พาลูกเล็กๆ เดินคุ้ยเขี่ยหาเมล็ดข้าวเปลือกและตัวแมลงตามพื้นดิน',
 'เห็นฝูงเป็ดว่ายน้ำดำน้ำหาหอยปลาตามคูคลอง เหล่านี้เป็นการหาอาหารตามธรรมชาติของสัตว์เลี้ยงของเรา',
 'แต่ก่อนนี้คนไทยเลี้ยงสัตว์ไว้เพียงเล็กน้อยเพื่อใช้งาน ชาวนา ชาวไร่เลี้ยงวัวควายไว้เพียงบ้านละสามหรือสี่ตัว สำหรับช่วยทำนา ทำไร่ เลี้ยงเป็ดเลี้ยงไก่ไว้เพียงฝูงเล็กๆ เพื่อกินไข่',
 'แต่ในปัจจุบันเราเลี้ยงสัตว์กันเป็นจำนวนมาก มีผู้เลี้ยงโคนมจำนวนสองถึงสามร้อยตัวในพื้นที่หลายร้อยไร่',
 'เพื่อรีดนมส่งเข้าโรงงานทำนมสดบรรจุกล่องนมผงและนมเม็ด มีการเลี้ยงไก่จำนวนนับพันตัว เพื่อเอาเนื้อส่งไปขายต่างประเทศ',
 '118',
 'การเลี้ยงสัตว์จำนวนมากๆ เช่นนี้ เราไม่ใช้วิธีปล่อยให้สัตว์หาอาหารกินเองตามธรรมชาติ',
 'แต่จะหาอาหารดีมีคุณภาพให้ เพื่อสัตว์เลี้ยงจะได้เติบโตเร็ว ไม่เป็นโรค และให้ผลิตผลสูง',
 'เราสามารถตรวจได้ว่า อาหารสัตว์ชนิดใดมีคุณภาพดีเหมาะสมกับสัตว์ และทำให้สัตว์เติบโตเร็วในชั่วระยะเวลาอันสั้น',
 'โดยใช้วิธีก

In [4]:
field = 'หากคุณรับน้องแมวมาเลี้ยงหรือมีคนให้น้องแมวมา คุณอาจจะไม่รู้ว่าเค้าอายุเท่าไร ซึ่งวิธีที่ง่ายที่สุดก็คือ พาน้องแมวไปหาสัตวแพทย์เพื่อให้คาดคะเนอายุให้ วันนี้เราลองมาดูกันว่าสัตวแพทย์มีวิธีคาดคะเนอายุน้องแมวอย่างไร เผื่อบางทีเราอาจจะคาดเดาอายุของน้องแมวแบบคร่าวๆได้ด้วยตัวเอง'

# Character Types

In [5]:
# t = match begining of the line
# q = optional
# Thai consonants พยัญชนะต้น
consonant = '[กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ]'
consonantq = '[กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ]?'
tconsonant = '^[กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ]'

# Final consonant (speller) ตัวสะกด
fconsonant = '[กขคฆงจชซญฎฏฐฑฒณดตถทธนบปพฟภมยรลวศษสฬอ]'
tfconsonant = '^[กขคฆงจชซญฎฏฐฑฒณดตถทธนบปพฟภมยรลวศษสฬอ]'

# Mixed cluster consonant พยัญชนะควบกล้ำทั้งหมด
consonantc = '[กขคจฉซตถทบปผพฟศส]'
tconsonantc = '^[กขคจฉซตถทบปผพฟศส]'

# Cluster consonant with lo พยัญชนะควบ "ล"
consonantl = '[กขคฉถบปผพฟ]'
tconsonantl = '^[กขคฉถบปผพฟ]'

# Cluster consonant with wo พยัญชนะควบ "ว"
consonantw = '[กขค]'
tconsonantw = '^[กขค]'

# Cluster consonant with ho พยัญชนะ "ห" นำ
consonanth = '[งญนมยรลว]'

# Final character in mae kor kar กก, กด, กบ, กง, กน, กม, + กอ
fkconsonant = '[กดบงนมอ]'

# Thai vowel character
vowel = '[ะาิีึืุูโเแัำไใฤฦ]'
vowelq = '[ะาิีึืุูโเแัำไใฤฦ]?'

# Thai tonal mark
tone = "[ ่ ้ ๋ ๊ ]".replace(" ", "")
toneq = '[ ่่ ้ ๋ ๊ ]?'. replace(" ", "")

# Thai number
number = '[๐๑๒๓๔๕๖๗๘๙]'

# Thai special character
special = '[็์ฯๆาฺํ()!?:;,.]'

# Character that must follow a character
strongbondchar = '[ะัาำิีึืฺุู็่้๊๋์]'
nstrongbondchar = '[^ะัาำิีึืฺุู็่้๊๋์]'

# Character that never used in start position (of word)
nonstartchar = '[ะัาำิีึืฺุูิี้่็๋๊ํ]'

# Character that never used in end position (of word)
nonendchar = '[เแโใไั]'

# Character that never used in the second position (of word)
nonsecondchar = '[ๆ์]'

# English Alphabet
alphabet = '[A-Za-z]'

# Number
number = '[0-9]'

# - Default Rule

In [6]:
# Rule d1 ; ฯลฯ
rule_d1 = r'ฯลฯ'

# Rule d2 ; Alphabet
rule_d2 = r'{}+'.format(alphabet)

# Rule d3 ; Number
rule_d3 = r'{}+'.format(number)

# Rule d4 ; Attach ร์, ดิ์, ตร์, ทธิ์, ถุ์ to the existing previous unit
rule_d4 = r'{}{}{}์'.format(consonant, consonantq, vowelq)

# Rule d5 ; Attach non-starting char to the existing previous unit
rule_d5 = r'{}'.format(nonstartchar)

# - Level 0 very risky rule, apply to all types of cluster 

In [7]:
# Rule l0.1 ใคร, ใกล้ ???? สา|เกล้|น|แก้ว, โคล้|อมวง
rule_l0_1 = r'[เแไใโ]{}[รลว]{}'.format(consonantc,toneq)

# Rule l0.2 ; ไหน, ไหล่
rule_l0_2 = r'[เแไใโ]ห{}{}'.format(consonanth, toneq)

# Rule l0.3 ; ครัว, ครั้ง ???? สิ|บวัน
rule_l0_3 = r'{}[รลว]ั{}{}'.format(consonantc,toneq,fconsonant)

# Rule l0.4 ; หนัง, หยั่ง
rule_l0_4 = r'ห{}ั{}{}'.format(consonanth, toneq,fconsonant)

# Rule l0.5 ; คลำ, ประ, ??? แบ|บระ|บบ, แบ|บระ|ดับ
rule_l0_5 = r'{}[รลว]{}[ะำ]'.format(consonantc,toneq)

# Rule l0.6 ; หนำ, หวะ
rule_l0_6 = r'ห{}{}[ะำ]'.format(consonanth, toneq)

# Rule l0.7 ; เสร็จ
rule_l0_7 = r'[เแ]{}[รลว]็{}'.format(consonantc,fconsonant)

# Rule l0.8 ; เหม็น
rule_l0_8 = r'[เแ]ห{}็{}'.format(consonanth,fconsonant)

# Rule l0.9 ; แคระ, โคร่ะ ???? แประ|ยะ
rule_l0_9 = r'[เแโ]{}[รลว]{}ะ'.format(consonantc, toneq)

# Rule l0.10 ; โหละ โหล่ะ
rule_l0_10 = r'[เแโ]ห{}{}ะ'.format(consonanth, toneq)

# Rule l0.11 ; เพราะ, เพร๋าะ
rule_l0_11 = r'เ{}[รลว]{}าะ'.format(consonantc, toneq)

# Rule l0.12 ; เหวาะ, เหร๊าะ
rule_l0_12 = r'เห{}{}าะ'.format(consonanth, toneq)

# Rule l0.13 ; เคว้า, เขย่า
rule_l0_13 = r'เ{}[รลว]{}า'.format(consonantc, toneq)

# Rule l0.14 ; เหว่า
rule_l0_14 = r'เห{}{}า'.format(consonanth, toneq)

# Rule l0.15 ; เกรอะ
rule_l0_15 = r'เ{}[รลว]{}อะ'.format(consonantc, toneq)

# Rule l0.16 ; เหล้อะ
rule_l0_16 = r'เห{}{}อะ'.format(consonanth, toneq)

# Rule l0.17 ; เคล๋ ????
rule_l0_17 = r'เ{}[รลว]{}'.format(consonantc, toneq)

# Rule l0.18 ; เหล๋
rule_l0_18 = r'เห{}{}'.format(consonanth, toneq)

# Rule l0.19 ; เคลียะ
rule_l0_19 = r'เ{}[รลว]ี{}ยะ'.format(consonantc, toneq)

# Rule l0.20 ; เหรียะ
rule_l0_20 = r'เห{}ี{}ยะ'.format(consonanth, toneq)

# Rule l0.21 ; เครือะ
rule_l0_21 = r'เ{}[รลว]ื{}อะ'.format(consonantc, toneq)

# Rule l0.22 ; เหรือะ
rule_l0_22 = r'เห{}ื{}อะ'.format(consonanth, toneq)

# - Level 1 cluster consonant may take additional final consonant.

In [8]:
# Rule l1.1 ; ??? เพล่ยั|ง
rule_l1_1 = r'เ{}[รลว]ี?{}ย'.format(consonantc, toneq)

# Rule l1.2 ; เหลี่ย|ม
rule_l1_2 = r'เห{}ี?{}ย'.format(consonanth, toneq)

# Rule l1.3 ; เคลือ เครอ
rule_l1_3 = r'เ{}[รลว]ื?{}อ'.format(consonantc, toneq)

# Rule l1.4 ; เหลือ
rule_l1_4 = r'เห{}ื?{}อ'.format(consonanth, toneq)

# Rule l1.5 ; ???? มา|กว่า
rule_l1_5 = r'{}[รลว]{}า'.format(consonantc,toneq)

# Rule l1.6 ; หนา, หน้า
rule_l1_6 = 'ห{}{}า'.format(consonanth,toneq)

# Rule l1.7 ; ????
rule_l1_7 = '{}[รลว][ิีุู]{}'.format(consonantc,toneq)

# Rule l1.8 ; หลี|อ
rule_l1_8 = 'ห{}[ิี]{}'.format(consonanth, toneq)

# - Level 2 never take any more final consonant.

In [9]:
# Rule 1 ; กัน
rule_1 = '{}ั{}{}'.format(consonant,toneq,fconsonant)

# Rule 1.1 ; ก็
rule_1_1 = '{}็'.format(consonant)

# Rule 1.2 ; กำ น่ะ
rule_1_2 = '{}{}[ะำ]'.format(consonant, toneq)

# Rule 2 ; กั๊วะ
rule_2 = '{}ั{}วะ'.format(consonant, toneq)

# Rule 3 ; เห็น
rule_3 = '[เแ]{}็{}'.format(consonant, fconsonant)

# Rule 4 ; โฮ๋ะ
rule_4 = '[เแโ]{}{}ะ'.format(consonant, toneq)

# Rule 5 ; เงาะ
rule_5 = 'เ{}{}าะ'.format(consonant, toneq)

# Rule 6 ; เงา
rule_6 = 'เ{}{}า'.format(consonant, toneq)

# Rule 7 ; เงอะ
rule_7 = 'เ{}{}อะ'.format(consonant, toneq)

# Rule 8 ; เงิก
rule_8 = 'เ{}ิ{}{}'.format(consonant, toneq, fconsonant)

# Rule 9 ; เงียะ
rule_9 = 'เ{}ี{}ยะ'.format(consonant, toneq)

# Rule 10 ; เงือะ 
rule_10 =  'เ{}ื{}อะ'.format(consonant, toneq)

# Rule 11 ; หวย ??? ค|ล่|อ|ง|แค|ล่วดี|
rule_11 = '{}{}ว{}'.format(consonant, toneq, fconsonant)

# Rule 12 ; กฤศ ??? ล|ดฤท|ธิ์|
rule_12 = '{}ฤ{}'.format(consonant, fconsonant)

# Rule 12.1 ; ให้
rule_12_1 = 'ใ{}{}'.format(consonant, toneq)

# Rule 16.1 ; หืม
rule_16_1 = '{}[ึื]{}{}'.format(consonant, toneq, fconsonant)

# - Level 3 may take additional final consonant.

In [40]:
# Rule 13 ; เมีย, เนย ??? เทยา
rule_13 = 'เ{}ี?{}ย'.format(consonant, toneq)

# Rule 14 ; เสือ, เออ ??? เข้อ้|าย
rule_14 = 'เ{}ื?{}อ'.format(consonant, toneq)

# Rule 14e ; เสือ
rule_14e = 'เ{}ื{}อ'.format(consonant, toneq)

# Rule 15 ; ง่า
rule_15 = '{}{}า'.format(consonant, toneq)

# Rule 16 ; งี่ งิ งุ งู
rule_16 = '{}[ิีุู]{}'.format(consonant, toneq)

# Rule 17 ; แง่
rule_17 = '[เแไโ]{}{}'.format(consonant, toneq)

# - Ad hoc rule

In [41]:
# Rule a1 ; อย่า ??? ห่|อยา|
rule_a1 = 'อย{}า'.format(toneq)

# Rule a2 ; อยู่ ??? ข้|อยุ|ติ
rule_a2 = 'อย[ูุ]{}'.format(toneq)

# Rule a3 ; หล่น
rule_a3 = 'ห{}{}{}'.format(consonanth, tone, fkconsonant)

# Rule a4 ; หน่วย
rule_a4 = 'ห{}{}ว{}'.format(consonanth, tone, fconsonant)

# Rule a5 ; ช่ง
rule_a5 = '{}{}{}'.format(consonant, tone, fkconsonant)
# Rule a6 ; อ้วน
rule_a6 = 'อ{}ว{}'.format(tone, fkconsonant)

# All TCC rules. Please Comment out risky rule

In [70]:
rules = {
    'd1': rule_d1, # ฯลฯ
    'd2': rule_d2, # Alphabet
    'd3': rule_d3, # Number
    'd4': rule_d4, # Attach ร์, ดิ์, ตร์, ทธิ์, ถุ์ to the existing previous unit
    'd5': rule_d5, # Attach non-starting char to the existing previous unit
    
#    'l0_1': rule_l0_1, # ใคร, ใกล้ ???? สา|เกล้|น|แก้ว, โคล้|อมวง
    'l0_2': rule_l0_2,  # ไหน, ไหล่
#    'l0_3': rule_l0_3, # ครัว, ครั้ง ???? สิ|บวัน
    'l0_4': rule_l0_4,  # หนัง, หยั่ง
#    'l0_5': rule_l0_5, # คลำ, ประ, ??? แบ|บระ|บบ, แบ|บระ|ดับ
    'l0_6': rule_l0_6,  # หนำ, หวะ
    'l0_7': rule_l0_7,  # เสร็จ
    'l0_8': rule_l0_8,  # เหม็น
#    'l0_9': rule_l0_9, # แคระ, โคร่ะ ???? แประ|ยะ
    'l0_10': rule_l0_10,
    'l0_11': rule_l0_11,
    'l0_12': rule_l0_12,
    'l0_13': rule_l0_13,
    'l0_14': rule_l0_14,
    'l0_15': rule_l0_15,
    'l0_16': rule_l0_16,
#    'l0_17': rule_l0_17,
    'l0_18': rule_l0_18,
    'l0_19': rule_l0_19,
    'l0_20': rule_l0_20,
    'l0_20': rule_l0_20,
    'l0_21': rule_l0_21,
    'l0_22': rule_l0_22,
    
    'l1_1': rule_l1_1,
    'l1_2': rule_l1_2,
#    'l1_3': rule_l1_3,
    'l1_4': rule_l1_4,
#    'l1_5': rule_l1_5,
    'l1_6': rule_l1_6,
#    'l1_7': rule_l1_7,
    'l1_8': rule_l1_8,
    
    '1': rule_1,
#    '1_1': rule_1_1, # เพล่่ยั|ง
    '1_2': rule_1_2,
    '2': rule_2,
    '3': rule_3,
    '4': rule_4,
    '5': rule_5,
    '6': rule_6,
    '7': rule_7,
    '8': rule_8,
    '9': rule_9,
    '10': rule_10,
#    '11': rule_11, ค|ล่|อ|ง|แค|ล่วดี|
#    '12': rule_12,
    '12_1': rule_12_1,
#    '16_1': rule_16_1,
    
#    '13': rule_13,
#    '14': rule_14,
    '14e': rule_14e,
    '15': rule_15,
    '16': rule_16,
    '17': rule_17,
    
#    'a1': rule_a1, # ห่|อยา|
#    'a2': rule_a2, # อยู่ ??? ข้|อยุ|ติ
    'a3': rule_a3,
    'a4': rule_a4,
#    'a5': rule_a5,
    'a6': rule_a6
}

In [71]:
test = "เพราะ, เพร๋าะ เหวาะ, เหร๊าะ, เหรือะ,เครือะ,เหลี่ยม เหล้ย, เครอ, เพื่อ"
re.findall(rule_14e, test)

['เพื่อ']

# Thai Character Cluster Function

In [72]:
def tcc_segment(field):
    '''Apply Thai Character Cluster segmentation to field'''
    output = []
    rule_list = []
    rlength = 1
    i = 0
    while i < len(field):
        for rid in rules:
            sub_field = field[i:]
            match = re.match(rules[rid], sub_field)
            if match:
                rule_list.append(rid)
                output.append(match.group())
                rlength = match.end()
                break
        if not match:
            rlength = 1
            rule_list.append("x")
            output.append(sub_field[0])
        i += rlength
        
    # join rule d5
    if len(output) <= 0:
        return []
    temp_output = [output[0]]
    for i in range(1, len(rule_list)):
        if rule_list[i] == "d5":
            temp_output[-1] += output[i]
        else:
            temp_output.append(output[i])
    output = temp_output
            
#    print("input: " ,field)
#    print("Output: ", end = " ")
#    for i, o in enumerate(output):
#        print('{}:{}|'.format(o,rule_list[i]), end="")
    #print("|".join(output))
    #print("|".join(rule_list))
    return output


# Test

In [73]:
error = 0

dataset = os.listdir('novel')

for data in dataset:
    sentences = []
    targets = []
    with open(os.path.join('novel', data), 'r') as file:
        for line in file:
            line = line.replace("\n","")
            sentences.append(line.replace("|",""))
            targets.append(line.split("|"))

    for ln, line in enumerate(sentences):
        i = 0 
        output = tcc_segment(sentences[ln])
#         try:
        for wid, target in enumerate(targets[ln]):
            combine = ""
            combine_debug = ""
            while len(target) > len(combine):
                combine += output[i]
                combine_debug += (output[i] + "|")
                i += 1
            if target != combine:
                print()
                print(targets[ln][wid-1] + target + targets[ln][wid+1])
                print("T->", target,"P->",combine_debug, flush=True)
                error += 1
                #print(data)
                break
#         except:
#             print('error')
#             continue
    print(".", end="")
print(error)


 14
T-> 1 P-> 14|

 60
T-> 6 P-> 60|

 60
T-> 6 P-> 60|

 60
T-> 6 P-> 60|

 60
T-> 6 P-> 60|

 16
T-> 1 P-> 16|

 12
T-> 1 P-> 12|

 16
T-> 1 P-> 16|

 16
T-> 1 P-> 16|

 32
T-> 3 P-> 32|

 12
T-> 1 P-> 12|

 50
T-> 5 P-> 50|

 12
T-> 1 P-> 12|

 12
T-> 1 P-> 12|

 60
T-> 6 P-> 60|

"32
T-> 3 P-> 32|

 17
T-> 1 P-> 17|

"10
T-> 1 P-> 10|

 12
T-> 1 P-> 12|

 12
T-> 1 P-> 12|

 50
T-> 5 P-> 50|

 10
T-> 1 P-> 10|

 15
T-> 1 P-> 150|
...
...RoadMap
T-> Road P-> RoadMap|
...............
จะมีเหตุผล
T-> มีเ P-> มี|เห|
..
ขี้โมโหมั่ง
T-> โมโห P-> โม|โหมั่|
....
คนโมโหร้าย
T-> โมโห P-> โม|โหร้า|
..
หรือหักเหวิถี
T-> หักเห P-> หัก|เหวิ|

 youd
T-> you P-> youd|

 wereare
T-> were P-> wereare|
.
นี่เจ็ก็
T-> เจ็ P-> เจ็ก็|

หายโมโหมั้ง
T-> โมโห P-> โม|โหมั้|
.................
จะโมโหร้าย
T-> โมโห P-> โม|โหร้า|

เรือเหลำ
T-> เห P-> เหลำ|
.
เป็นหนี้
T-> ห P-> หนี้|
......
เป็นหนี้
T-> ห P-> หนี้|

เป็นหนี้
T-> ห P-> หนี้|
........
<NE>มิ่งขวัญ</NE>มีเหตุผล
T-> มีเ P-> มี|เห|

ใช้หนี้
T-> ห P-> ห